In [2]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

from antipark import *
from antipark.models import Product, Category

In [9]:
import json

with open('../antipark/db.json', encoding='utf=8', mode='r') as fo:
    data = json.load(fo)

d = {}
for k,v in data['products'].items():
    d[k] = v

In [4]:
db.create_all()

2016-11-04 22:13:03,225 INFO sqlalchemy.engine.base.Engine DESCRIBE `services`
2016-11-04 22:13:03,226 INFO sqlalchemy.engine.base.Engine {}
2016-11-04 22:13:03,234 INFO sqlalchemy.engine.base.Engine DESCRIBE `categories`
2016-11-04 22:13:03,234 INFO sqlalchemy.engine.base.Engine {}
2016-11-04 22:13:03,242 INFO sqlalchemy.engine.base.Engine DESCRIBE `products`
2016-11-04 22:13:03,243 INFO sqlalchemy.engine.base.Engine {}


In [29]:
# if smth was wrong
# db.session.rollback()

In [5]:
for category, values in d.items():
    
    description = values['desc']
    
    cat = Category(category=category,
                   description=description,
                  )
    db.session.add(cat)
    db.session.commit()

2016-09-14 18:34:41,966 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2016-09-14 18:34:41,973 INFO sqlalchemy.engine.base.Engine INSERT INTO categories (category, description) VALUES (%(category)s, %(description)s)
2016-09-14 18:34:41,976 INFO sqlalchemy.engine.base.Engine {'description': 'Парковочные столбики предназначены для ограждения территории от нежелательного проезда автотранспорта. В последнее время в городе существенно увеличивается количество автомобилей. Владельцы транспортных средств любят парковать своих «железных коней» на газонах, пешеходных зонах и тротуарах. Поэтому  столбики часто устанавливают на тротуары по самому краю дороги, для того чтобы предотвратить нежелательную парковку автомобилей и обеспечить проход граждан. Дорожные столбики подразделяются на бетонируемые, анкерные, передвижные, складные и съемные.Бетонируемые столбы крепятся к асфальту или грунту при помощи бетонирования:  сначала в асфальте делается отверстие 30х30см, после чего установленный в н

In [5]:
categories = Category.query.all()
categories

2016-11-04 22:13:16,727 INFO sqlalchemy.engine.base.Engine SELECT categories.id AS categories_id, categories.category AS categories_category, categories.description AS categories_description 
FROM categories
2016-11-04 22:13:16,728 INFO sqlalchemy.engine.base.Engine {}


In [13]:
from collections import namedtuple

Cat = namedtuple('Cat', ['category', 'idx'])
cats = []
for category in categories:
#     if category.category == 'Знаки дорожные':
#         continue
    cat = Cat(category.category, category.id)
    cats.append(cat)
    
cats

[Cat(category='Столбики парковочные', idx=3),
 Cat(category='Конусы', idx=4),
 Cat(category='Бетонные урны', idx=5),
 Cat(category='Автоматические шлагбаумы', idx=6),
 Cat(category='Пластиковые сетки', idx=7),
 Cat(category='Бетонные полусферы', idx=8),
 Cat(category='Барьеры', idx=9),
 Cat(category='Турникеты', idx=10),
 Cat(category='Лежачий полицейский', idx=11),
 Cat(category='Бетонные вазоны', idx=12),
 Cat(category='Бетонные скамейки', idx=13),
 Cat(category='Оградительная лента', idx=14),
 Cat(category='Приводы для ворот', idx=15),
 Cat(category='Знаки дорожные', idx=16)]

In [14]:
for category, values in d.items():
    
    category_id = None
    for cat in cats:
        if category == cat.category:
            category_id = cat.idx
            break
    if not category_id:
        continue
    
    all_vars = values['variations']
    
    for var in all_vars:
        # prices
        prices = all_vars[var]['prices'].strip('[]').split(', ')
            
        # stage_prices
        stage_prices = all_vars[var]['stage'].strip('[]').split(', ')
            
        # names
        names = all_vars[var]['names'].strip('[]').split(', ')
        
        # subcategory
        subcategory = all_vars[var]['general']
        
        # specs
        chars = all_vars[var]['chars'].strip('[]').split(', ')
        char_values_str = all_vars[var]['values'].split('], [')
        char_values_temp = []
        for ch_v in char_values_str:
            char_values_temp.append(ch_v.strip('[]').split(', '))
                
        specs = []
        print(chars)
        for ch_v in char_values_temp:
            print(ch_v)
            print()
            char_values_dict = {}
            for i, ch in enumerate(ch_v):
                char_values_dict[chars[i]] = ch
            specs.append(str(char_values_dict))
            
        # images`
        images_str = all_vars[var]['imgs'].strip('[]').split(', ')
        images = []
        for img in images_str:
            image = all_vars[var]['dest'] + img + ".jpg"
            images.append(image)
            
        
        # commit rec to db
        for i, title in enumerate(names):
            prod = Product(category=category_id,
                           subcategory=subcategory,
                           title=title,
                           specs=specs[i],
                           price=prices[i],
                           stage_price=stage_prices[i],
                           image=images[i],
                           )
            db.session.add(prod)
            db.session.commit()

['Размеры', 'Вес']
['400х400х400', '85кг']

['800х800х420', '190кг']

['700х700х420', '125кг']

['1200х350х400', '170кг']

['750х750х420', '205кг']

['750х750х420', '205кг']

['750х750х420', '205кг']

['750х750х420', '205кг']

['750х750х420', '205кг']

['1200х350х400', '170кг']

['1200х350х400', '170кг']

['1200х350х400', '170кг']

2016-11-04 23:00:50,661 INFO sqlalchemy.engine.base.Engine INSERT INTO products (category, subcategory, title, specs, price, stage_price, image) VALUES (%(category)s, %(subcategory)s, %(title)s, %(specs)s, %(price)s, %(stage_price)s, %(image)s)
2016-11-04 23:00:50,662 INFO sqlalchemy.engine.base.Engine {'price': '1299', 'subcategory': '', 'category': 12, 'title': 'Вазон бетонный уличный квадратный', 'image': 'vazoni/square.jpg', 'stage_price': '-', 'specs': "{'Вес': '85кг', 'Размеры': '400х400х400'}"}
2016-11-04 23:00:50,852 INFO sqlalchemy.engine.base.Engine COMMIT
2016-11-04 23:00:50,896 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2016-11-04 23:00:

In [12]:
for category in categories:
    if category.category != 'Знаки дорожные':
        continue
    cat_znaki = Cat(category.category, category.id)
    
cat_znaki

Cat(category='Знаки дорожные', idx=16)

In [ ]:
for category, values in d.items():
    
    if category != 'Знаки дорожные':
        continue
    
    category_id = 16
    
    all_vars = values['variations']
    
    for var in all_vars:
        # names
        names = all_vars[var]['names'].strip('[]').split(', ')
        
        # specs
        chars = all_vars[var]['chars'].strip('[]').split(', ')
        char_values_str = all_vars[var]['values'].split('], [')
        char_values_temp = []
        for ch_v in char_values_str:
            char_values_temp.append(ch_v.strip('[]').split(', '))
        
        # spec_values
        values = all_vars[var]['values'].strip('[]').split(', ')        
                
        specs = []
        print(chars)
        for ch_v in char_values_temp:
            print(ch_v)
            print()
            char_values_dict = {}
            for i, ch in enumerate(ch_v):
                char_values_dict[chars[i]] = ch
            specs.append(str(char_values_dict))
            
        # images
        images_str = all_vars[var]['imgs'].strip('[]').split(', ')
        images = []
        for img in images_str:
            image = all_vars[var]['dest'] + img + ".jpg"
            images.append(image)
            
        
        # commit rec to db
        for i, title in enumerate(names):
            prod = Product(category=category_id,
                           subcategory=subcategory,
                           title=title,
                           specs=specs[i],
                           price=prices[i],
                           stage_price=stage_prices[i],
                           image=images[i],
                           )
            db.session.add(prod)
            db.session.commit()